In [1]:
import pygame
import moderngl
import numpy as np
import glm
from LoadObject import getObjectData

pygame 2.6.0 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Initialize Pygame
pygame.init()

# Set OpenGL version and create an OpenGL context
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MAJOR_VERSION, 4)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MINOR_VERSION, 1)
pygame.display.set_mode((800, 600), pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)

# Get the ModernGL context
gl = moderngl.create_context()
gl.enable(moderngl.DEPTH_TEST)

# Dictionary mapping keys 1-6 to corresponding OBJ files
obj_files = {
    pygame.K_1: "Teapot.obj",
    pygame.K_2: "Tetrahedron.obj",
    pygame.K_3: "Hexahedron.obj",
    pygame.K_4: "Octahedron.obj",
    pygame.K_5: "Dodecahedron.obj",
    pygame.K_6: "Icosahedron.obj"
}

2024-09-24 00:09:51.508 Python[5148:1903539] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [3]:
# Function to load an object and return its VAO and transformation matrix
def load_object(obj_file):
    vertexData, bounds = getObjectData(obj_file, normal=True)
    
    # Create new buffer with the object vertex data
    vertexBuffer = gl.buffer(vertexData.astype('f4').tobytes())
    
    # Create a new vertex array object (VAO)
    renderable = gl.vertex_array(
        shaderProgram, 
        [(vertexBuffer, '3f 3f', 'in_position', 'in_normal')]
    )
    
    return renderable, bounds

In [4]:
# Vertex Shader
vertex_shader_code = '''
#version 330 core

layout (location = 0) in vec3 in_position;
layout (location = 1) in vec3 in_normal;

uniform mat4 view;
uniform mat4 perspective;

out vec3 f_normal;

void main() {
    f_normal = in_normal;
    gl_Position = perspective * view * vec4(in_position, 1.0);
}
'''

In [5]:
# Fragment Shader
fragment_shader_code = '''
#version 330 core

in vec3 f_normal;
out vec4 out_color;

void main() {
    out_color = vec4(0.5 * (normalize(f_normal) + 1.0), 1.0);
}
'''

In [6]:
# Compile shaders and link them into a program
shaderProgram = gl.program(
    vertex_shader=vertex_shader_code,
    fragment_shader=fragment_shader_code
)

In [7]:
# Load all objects at the start and store them
loaded_objects = {}
for key, obj_file in obj_files.items():
    renderable, bounds = load_object(obj_file)
    loaded_objects[key] = (renderable, bounds)

Normal exists
Normal computed.
Normal computed.
Normal computed.
Normal computed.
Normal computed.


In [8]:
# Set the initial object (Teapot.obj)
current_key = pygame.K_1  # Start with key 1 (Teapot.obj)
renderable, bounds = loaded_objects[current_key]

# Pygame window setup
pygame.display.set_caption("Assignment 5: Nelson Herrera")

# Camera setup
def compute_camera_position(angle, radius):
    # Compute unit vector that makes a 45-degree angle to Y-axis in the XY-plane
    x = np.cos(angle) * np.sqrt(2) / 2
    y = np.sqrt(2) / 2
    z = np.sin(angle) * np.sqrt(2) / 2
    
    # Return the eye position based on radius distance away from the object's center
    return glm.vec3(x, y, z) * radius * 2

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
            running = False
        
        # Check if a number key between 1-6 is pressed
        if event.type == pygame.KEYDOWN and event.key in obj_files:
            current_key = event.key
            renderable, bounds = loaded_objects[current_key]

        # Handle window resize
        if event.type == pygame.WINDOWRESIZED:
            width, height = event.x, event.y
            gl.viewport = (0, 0, width, height)
    
    # Get the elapsed time for orbiting
    time_in_seconds = pygame.time.get_ticks() / 1000.0
    
    # Compute the perspective matrix
    aspect_ratio = 800 / 600  # Width/height of your window (default values)
    fov = 60
    near_plane =  0.1  # Near distance
    far_plane = 100 # Far distance
    perspective_matrix = glm.perspective(glm.radians(fov), aspect_ratio, near_plane, far_plane)
    eye_point = glm.vec3(1,1,0)
    normalized_eye_point = glm.normalize(eye_point) + glm.vec3(bounds.center)

    view_matrix = glm.lookAt(normalized_eye_point * bounds.radius, glm.vec3(bounds.center), glm.vec3(0,1,0))
    
    # # Upload the view and perspective matrices to the shader
    shaderProgram['view'].write(view_matrix)
    shaderProgram['perspective'].write(perspective_matrix)

    #   # Upload the matrices to the shader
    # shaderProgram['view'].write(np.array(view_matrix, dtype='f4').tobytes())
    # shaderProgram['perspective'].write(np.array(perspective_matrix, dtype='f4').tobytes())


    # Clear the screen
    gl.clear(0.2, 0.2, 0.2)

    # Render the object
    renderable.render(moderngl.TRIANGLES)

    # Swap buffers
    pygame.display.flip()

# Cleanup
pygame.quit()
quit()